# Stackstrom Introduction

This notebook will walk you through some of the basics of stackstorm. This notebook has been developed to accomodate mutiple users logging into the same stackstorm server. This is not by any means a super deep dive but hopefully will explain why stackstorm should be in your automation toolbag. Run the following cell to see an overview of the different components.

# WARNING! 
To learn stackstorm we have to start at the bottom. Bear with me to the end and we can explore stackstorm a different way.

<img src="http://www.techworldwookie.com/images/st2-overview.jpg" width="600" height="600">

StackStrom is a automation framework that uses **Sensors** to watch and listen to alarm/event queues. **Triggers** to tell **Rules** when to launch, and **Actions/Workflows** to change the existing state.

A quick check of the StackStorm (st2) version.

In [39]:
%%bash
st2 --version

st2 3.1.0, on Python 3.6.8


Getting help with st2

In [3]:
%%bash
st2 -h

usage: st2 [-h] [--version] [--url BASE_URL] [--auth-url AUTH_URL]
           [--api-url API_URL] [--stream-url STREAM_URL]
           [--api-version API_VERSION] [--cacert CACERT]
           [--config-file CONFIG_FILE] [--print-config] [--skip-config]
           [--debug]
           {run,action,action-alias,auth,login,whoami,apikey,execution,inquiry,key,pack,policy,policy-type,rule,webhook,timer,runner,sensor,trace,trigger,trigger-instance,rule-enforcement,workflow,service-registry,role,role-assignment}
           ...

CLI for StackStorm event-driven automation platform. https://stackstorm.com

positional arguments:
  {run,action,action-alias,auth,login,whoami,apikey,execution,inquiry,key,pack,policy,policy-type,rule,webhook,timer,runner,sensor,trace,trigger,trigger-instance,rule-enforcement,workflow,service-registry,role,role-assignment}
    run                 Invoke an action manually.
    action              An activity that happens as a response to the external
                  

Stackstorm is a platform that you will need to sign in to. Use the installation password.Change password if needed.

In [40]:
%%bash
st2 login st2admin --password "siesta3"

Logged in as st2admin

Note: You didn't use --write-password option so the password hasn't been stored in the client config and you will need to login again in 24 hours when the auth token expires.
As an alternative, you can run st2 login command with the "--write-password" flag, but keep it mind this will cause it to store the password in plain-text in the client config file (~/.st2/config).


Actions are the workhorse of stackstorm. From the diagram above they are the last thing that gets called to kick off some form of automation. To see what actions you have installed on your st2 server run this.

In [41]:
%%bash
st2 action list

+----------------------------------+---------------+----------------------------------------------------+
| ref                              | pack          | description                                        |
+----------------------------------+---------------+----------------------------------------------------+
| arubacx.get_system               | arubacx       | Get a dump of system information from Aruba CX     |
|                                  |               | switch                                             |
| arubacx.get_vlans                | arubacx       | Get a python dictionary of vlans from an Aruba CX  |
|                                  |               | switch                                             |
| arubacx.post_vlans               | arubacx       | Add vlans to an Aruba CX switch                    |
| chatops.format_execution_result  | chatops       | Format an execution result for chatops             |
| chatops.match                    | chatops  

Thats quite a few packs and actions. We can narrow down our search using the **-p** flag and specifying the pack we are interested in.

In [42]:
%%bash
st2 action list -p core

+---------------------+------+----------------------------------------------------+
| ref                 | pack | description                                        |
+---------------------+------+----------------------------------------------------+
| core.announcement   | core | Action that broadcasts the announcement to all     |
|                     |      | stream consumers.                                  |
| core.ask            | core | Action for initiating an Inquiry (usually in a     |
|                     |      | workflow)                                          |
| core.echo           | core | Action that executes the Linux echo command on the |
|                     |      | localhost.                                         |
| core.http           | core | Action that performs an http request.              |
| core.inject_trigger | core | Action which injects a new trigger in the system.  |
| core.local          | core | Action that executes an arbitrary Linux comma

Additionally, you can look at sensors, rules, and triggers using the same syntax. Results will vary depending on what is installed on the st2 server.

In [43]:
%%bash
st2 sensor list

+-----------------------------+---------+---------------------------------------+---------+
| ref                         | pack    | description                           | enabled |
+-----------------------------+---------+---------------------------------------+---------+
| linux.FileWatchSensor       | linux   | Sensor which monitors files for new   | True    |
|                             |         | lines                                 |         |
| twitter.TwitterSearchSensor | twitter | Sensor which monitors twitter         | True    |
|                             |         | timeline for new tweets matching the  |         |
|                             |         | specified criteria                    |         |
| twitter.TwitterStreamSensor | twitter | Sensor which monitors twitter         | True    |
|                             |         | timeline for new tweets matching the  |         |
|                             |         | specified criteria                    

In [44]:
%%bash
st2 rule list

+---------------------------------+---------------+---------------------------------------+---------+
| ref                             | pack          | description                           | enabled |
+---------------------------------+---------------+---------------------------------------+---------+
| chatops.notify                  | chatops       | Notification rule to send results of  | True    |
|                                 |               | action executions to stream for       |         |
|                                 |               | chatops                               |         |
| orange.process_trump_tweet_rule | orange        | If tweet is about a witch hunt,       | True    |
|                                 |               | process it.                           |         |
| stackstormdwr.hpecfm-alarms     | stackstormdwr | Get HPECFM alarms and load mongo      | True    |
|                                 |               | databases.                    

In [45]:
%%bash
st2 trigger list

+--------------------------------------+---------+----------------------------------------------------+
| ref                                  | pack    | description                                        |
+--------------------------------------+---------+----------------------------------------------------+
| core.st2.CronTimer                   | core    | Triggers whenever current time matches the         |
|                                      |         | specified time constaints like a UNIX cron         |
|                                      |         | scheduler.                                         |
| core.st2.DateTimer                   | core    | Triggers exactly once when the current time        |
|                                      |         | matches the specified time. e.g. timezone:UTC      |
|                                      |         | date:2014-12-31 23:59:59.                          |
| core.st2.IntervalTimer               | core    | Triggers on s

You can also have st2 run commands either **locally or remotely**. As you noted from above, st2 has CORE actions that make this easy. This means any command you can run from the command line can be executed in st2, either from the local host or any remote host.
You will see the results of the command returned in stdout (standard out) and the command itself was successful.

In [46]:
%%bash
st2 run core.local -- date -R

.
id: 5efb9dbd5f627dead071cc47
status: succeeded
parameters: 
  cmd: date -R
result: 
  failed: false
  return_code: 0
  stderr: ''
  stdout: Tue, 30 Jun 2020 13:17:01 -0700
  succeeded: true


Notice the 'id' number? Everytime st2 executes a task, it tracks the outcome in the executions list. You can view that list by running the below command.

In [47]:
%%bash
st2 execution list

+--------------------------+----------------------+--------------+-------------------------+----------------------+------------------+
| id                       | action.ref           | context.user | status                  | start_timestamp      | end_timestamp    |
+--------------------------+----------------------+--------------+-------------------------+----------------------+------------------+
| 5efa61145f627d03cc6ccec7 | stackstormdwr.get-   | stanley      | failed (26s elapsed)    | Mon, 29 Jun 2020     | Mon, 29 Jun 2020 |
|                          | hpeov-alarms         |              |                         | 21:45:56 UTC         | 21:46:22 UTC     |
| 5efa623f5f627d03cc6ccf07 | stackstormdwr.get-   | stanley      | failed (68s elapsed)    | Mon, 29 Jun 2020     | Mon, 29 Jun 2020 |
|                          | hpecfm-alarms        |              |                         | 21:50:55 UTC         | 21:52:03 UTC     |
| 5efa62405f627d03cc6ccf0a | stackstormdwr.get-   | sta

+------------------------------------------------------------------------------------------------------------------------------------+
| Note: Only first 50 action executions are displayed. Use -n/--last flag for more results.                                          |
+------------------------------------------------------------------------------------------------------------------------------------+


### TRY ON YOUR OWN: 
Every one of the 'id's' will have all the the errors, stdin, stdout, parameters and return code. This is super handy when trying to debug any actions you are creating. You would use the **st2 execution get (id)** command. Since these numbers are randomly generated you will have to do this command on your own. Just copy and paste any **id** at the end of the st2 execution get command.

In [ ]:
%%bash 
st2 execution get (paste any id here)

## NOTE:
When manually running an action from the st2 command line, you need to include the pack name, where the action lives, and the name of the action. **st2 run pack.action**. Actions may or may not require extra parameters to function. To learn what any action requires we need to look at the internals of the action by using the **get** command. Let's look at the core pack and what is required for the echo action. Not all parameters are required, look carefully. If the parameter has a default setting, you don't have to include it.

In [48]:
%%bash
st2 action get core.echo

+---------------+--------------------------------------------------------------+
| Property      | Value                                                        |
+---------------+--------------------------------------------------------------+
| id            | 5d94f2995f627d6afd533df3                                     |
| uid           | action:core:echo                                             |
| ref           | core.echo                                                    |
| pack          | core                                                         |
| name          | echo                                                         |
| description   | Action that executes the Linux echo command on the           |
|               | localhost.                                                   |
| enabled       | True                                                         |
| entry_point   |                                                              |
| runner_type   | local-shel

Looks like we need to add the parameter message to our call for action. So we add message='our string'.

In [49]:
%%bash
st2 run core.echo message='Lorem ipsum, blah, blah, blah'

.
id: 5efb9e1b5f627dead071cc4a
status: succeeded
parameters: 
  message: Lorem ipsum, blah, blah, blah
result: 
  failed: false
  return_code: 0
  stderr: ''
  stdout: Lorem ipsum, blah, blah, blah
  succeeded: true


OK, now let's look at how we will run something a little more useful.Let's list all the files in the directory.

In [50]:
%%bash
st2 run core.local cmd='ls'

.
id: 5efb9e285f627dead071cc4d
status: succeeded
parameters: 
  cmd: ls
result: 
  failed: false
  return_code: 0
  stderr: ''
  stdout: 'bin
    boot
    cdrom
    dev
    etc
    home
    initrd.img
    initrd.img.old
    lib
    lib64
    lost+found
    media
    mnt
    opt
    proc
    root
    run
    sbin
    snap
    srv
    swapfile
    sys
    tmp
    usr
    var
    vmlinuz
    vmlinuz.old'
  succeeded: true


Maybe you want to scrape some information from a web server, we can use this!

In [51]:
%%bash
st2 run core.http url="http://www.techworldwookie.com"

.
id: 5efb9e365f627dead071cc50
status: succeeded
parameters: 
  url: http://www.techworldwookie.com
result: 
  body: "<!DOCTYPE html>

<html lang="en-US">

<head>

\t<meta charset="UTF-8">

\t<meta name="viewport" content="width=device-width, initial-scale=1">

\t<link rel="profile" href="http://gmpg.org/xfn/11">

\t<link rel="pingback" href="http://www.techworldwookie.com/xmlrpc.php">

\t<title>techworldwookie &#8211; A curious wookie explores teknology</title>
<link rel='dns-prefetch' href='//fonts.googleapis.com' />
<link rel='dns-prefetch' href='//s.w.org' />
<link rel="alternate" type="application/rss+xml" title="techworldwookie &raquo; Feed" href="https://www.techworldwookie.com/feed/" />
<link rel="alternate" type="application/rss+xml" title="techworldwookie &raquo; Comments Feed" href="https://www.techworldwookie.com/comments/feed/" />
\t\t<script type="text/javascript">
\t\t\twindow._wpemojiSettings = {"baseUrl":"https:\\/\\/s.w.org\\/images\\/core\\/emoji\\/12.0.0-1\\/72x72\\

## Custom Actions

Let's run a custom action. nasa_apod is an action that will get the NASA picture of the day for NASA's server and return some information about the picture and the URL. This action is part of a stackstorm integration pack called tektalk. Let's see if it is installed.

In [52]:
%%bash
st2 pack list

+---------------+---------------+--------------------------------+---------+------------------+
| ref           | name          | description                    | version | author           |
+---------------+---------------+--------------------------------+---------+------------------+
| arubacx       | arubacx       | Integration pack for Aruba OS- | 0.1.0   | Rick Kauffman    |
|               |               | CX                             |         |                  |
| chatops       | chatops       | ChatOps integration pack       | 3.1.0   | StackStorm, Inc. |
| core          | core          | Basic core actions.            | 3.1.0   | StackStorm, Inc. |
| default       | default       | Default pack where all         | 3.1.0   | StackStorm, Inc. |
|               |               | resources created using the    |         |                  |
|               |               | API with no pack specified get |         |                  |
|               |               | saved.

Yes, we see the pack **tektalk** listed, now let's see what actions are made available to us.

In [53]:
%%bash
st2 action list -p tektalk

+----------------------------+---------+----------------------------------------------------+
| ref                        | pack    | description                                        |
+----------------------------+---------+----------------------------------------------------+
| tektalk.nasa_apod          | tektalk | Queries NASA's APOD (Astronomy Picture Of the Day) |
|                            |         | API to get the link to the picture of the day.     |
| tektalk.nasa_apod_workflow | tektalk | A workflow that runs a series of actions           |
+----------------------------+---------+----------------------------------------------------+


Great, now we have to see what paramerters are needed for the action to run.

In [54]:
%%bash
st2 action get tektalk.nasa_apod

+---------------+--------------------------------------------------------------+
| Property      | Value                                                        |
+---------------+--------------------------------------------------------------+
| id            | 5efa47295f627da0ad420701                                     |
| uid           | action:tektalk:nasa_apod                                     |
| ref           | tektalk.nasa_apod                                            |
| pack          | tektalk                                                      |
| name          | nasa_apod                                                    |
| description   | Queries NASA's APOD (Astronomy Picture Of the Day) API to    |
|               | get the link to the picture of the day.                      |
| enabled       | True                                                         |
| entry_point   | nasa_apod.py                                                 |
| runner_type   | python-scr

**API_KEY** and **HD** are defaulted. The only parameter we need to supply is the date for the picture we want. It will need to be in a **yyyy-mm-dd** format.

### Action files
Actions are made up of two different files. An introductory YAML file and a script of some kind. Action files live in the **/actions** directory. Let's take a look.

In [55]:
%%bash
ls -lia /opt/stackstorm/packs/tektalk/actions 

total 28
674833 drwxrwxr-x 4 root st2packs 4096 Jun 29 13:13 .
670610 drwxrwxr-x 8 root st2packs 4096 Jun 29 12:54 ..
674834 -rwxrwxr-x 1 root st2packs    0 Jun 29 12:54 .gitkeep
674835 -rwxrwxr-x 1 root st2packs  662 Jun 29 12:54 nasa_apod.py
674837 -rwxrwxr-x 1 root st2packs  181 Jun 29 12:54 nasa_apod_workflow.yaml
674836 -rwxrwxr-x 1 root st2packs  527 Jun 29 12:54 nasa_apod.yaml
670609 drwxrwxr-x 2 root st2packs 4096 Jun 29 13:13 __pycache__
674838 drwxrwxr-x 2 root st2packs 4096 Jun 29 12:54 workflows


Let's look at the introductory YAML file

In [56]:
%%bash
cat /opt/stackstorm/packs/tektalk/actions/nasa_apod.yaml

---
name: nasa_apod
pack: tektalk
description: "Queries NASA's APOD (Astronomy Picture Of the Day) API to get the link to the picture of the day."
runner_type: "python-script"
enabled: true
entry_point: nasa_apod.py
parameters:
  api_key:
    type: string
    description: "API key to use for api.nasa.gov."
    default: "DEMO_KEY"
  hd:
    type: boolean
    description: "Retrieve the high resolution image."
    default: false
  date:
    type: string
    description: "The date [YYYY-MM-DD] of the APOD image to retrieve."


This is a **YAML** file so it will start with three dashes. We have the name of the action and the name of the pack it resides. The obilatory description and then the runner_type. There are **13 different types** of runners including WinRM Powershell. IN our example we will be using a python script, **nasa_apod.py**. The file then informs the python script that it will be receiveing three parameters. Let's look at the python script.

In [57]:
%%bash
cat /opt/stackstorm/packs/tektalk/actions/nasa_apod.py

#!/usr/bin/env python
#
# Description:
#   Queries NASA's APOD (Astronomy Picture Of the Day) API to get the link to the picture
#   of the day.
#
import json
import requests
from st2common.runners.base_action import Action

API_URL = "https://api.nasa.gov/planetary/apod"


class Apod(Action):

    def run(self, api_key, date, hd):
        params = {'api_key': api_key,
                  'hd': hd}
        if date is not None:
            params['date'] = date

        response = requests.get(API_URL, params=params)
        response.raise_for_status()
        data = response.json()
        if hd:
            data['url'] = data['hdurl']
        return data


We pass the api_key, date and hd variable to the python script and use **requests** to get the response. The python script returns the variable **data**. Let's run the action like this!

In [58]:
%%bash
st2 run tektalk.nasa_apod date=2018-07-04

.
id: 5efb9e975f627dead071cc53
status: succeeded
parameters: 
  date: '2018-07-04'
result: 
  exit_code: 0
  result:
    copyright: Michael Seeley
    date: '2018-07-04'
    explanation: If you saw the dawn's early light from Cape Canaveral Air Force Station last Friday, June 29, then you could have seen this rocket's red glare. The single 277-second long exposure, made from the roof of NASA's Vehicle Assembly building, shows a predawn Falcon 9 launch, the rocket streaking eastward into the sky about 45 minutes before sunrise. At high altitude, its stage separation plume is brightly lit by the Sun still below the eastern horizon. The Falcon 9 rocket's first stage had been launched before, lofting the Transiting Exoplanet Survey Satellite (TESS) into orbit on April 18, only 72 days earlier. For this launch of SpaceX Commercial Resupply Service mission 15 (CRS-15) it carried an also previously flown Dragon capsule. But no further reuse of this Falcon 9 was planned so no dramatic first st

### Hip Tip Of The Week!:
It is very easy to take an existing shell script or python script and convert it to be used with stackstorm. You do not need to rewrite all of your automation scripts. 

## Workflows

Actions by themselves are basically just more of the same. They are just scripts. Workflows allow us to string several actions together and feed the output of one action into another. Wrokflows, like actions require an intrductory YAML file and then another YAML file in the workflows directory. Workflows can call on any action from any st2 pack. The new workflow system for st2 is **Orquesta**, let look at our example workflow. First the introductory YAML.

In [59]:
%%bash
cat /opt/stackstorm/packs/tektalk/actions/nasa_apod_workflow.yaml

---
name: nasa_apod_workflow
pack: tektalk
description: A workflow that runs a series of actions
runner_type: orquesta
entry_point: workflows/nasa_apod_workflow.yaml
enabled: true



The key takeaway is the runner_type is now Orquesta. Notice that the entry__point is another YAML with the exact same name located in the workflows directory. Let's look at that file.

In [60]:
%%bash
cat /opt/stackstorm/packs/tektalk/actions/workflows/nasa_apod_workflow.yaml

version: '1.0'

description: A workflow that gets the apod and echos results.

tasks:
  get_apod_url:
    action: tektalk.nasa_apod
    input:
      date: <%now().format("%Y-%m-%d")%>
    next:
      - when: <% succeeded() %>
        publish:
          - apod: <% result().stdout %>
        do: echo_results

  echo_results:
    action: core.echo
    input:
      message: "<% ctx().apod %>"
    next: 
      - when: <% succeeded() %>
        do: uname

  uname: 
    action: core.local
    input:
      cmd: "uname -a"
  

Let's break this down. This is just a list of tasks. Tasks are carried out by **actions**. Actions get parameters (or not). Results are **published** to the **context** where they can be reused by other actions.

# The Elephant in the room
What we see here is not python. It is YAML (Yet Another Markup Language). I just takes a few lines of these YAML files to stich together a very complex workflows. Calling any actions from any pack is quite powerful. Using stackstorm will save you from writing thousands of lines of python. 

You would not normally run a workflow manually, that's really not automation becasue a human is involved. Workflows are assigned to st2 rules that get activated by triggers. That's the magic of st2. In just this one special case however, we will manuall run our workflow and see waht goes on behind the scenes.

In [61]:
%%bash
st2 run tektalk.nasa_apod_workflow

..
id: 5efba1bb5f627dead071cc56
action.ref: tektalk.nasa_apod_workflow
parameters: None
status: succeeded
start_timestamp: Tue, 30 Jun 2020 20:34:03 UTC
end_timestamp: Tue, 30 Jun 2020 20:34:07 UTC
result: 
  output: null
+--------------------------+------------------------+--------------+-------------------+--------------------------+
| id                       | status                 | task         | action            | start_timestamp          |
+--------------------------+------------------------+--------------+-------------------+--------------------------+
| 5efba1bc5f627d03cba69578 | succeeded (1s elapsed) | get_apod_url | tektalk.nasa_apod | Tue, 30 Jun 2020         |
|                          |                        |              |                   | 20:34:04 UTC             |
| 5efba1bd5f627d03cba6957b | succeeded (1s elapsed) | echo_results | core.echo         | Tue, 30 Jun 2020         |
|                          |                        |              |              

So what happened? Our workflow had three tasks, get a picture from a webserver, echo the results, and tell us what linux we are ruinning. St2 informs us that all three tasks were executed successfully. If you are curious about any of these executions, remember st2 lets us look at each one in greater detail with copy and pasting the **execution id** in the following command.

In [62]:
%%bash 
st2 execution get paste-any-id-here

id: 5efba1be5f627d03cba6957e
status: succeeded (0s elapsed)
parameters: 
  cmd: uname -a
result: 
  failed: false
  return_code: 0
  stderr: ''
  stdout: 'Linux ubuntu 5.3.0-40-generic #32~18.04.1-Ubuntu SMP Mon Feb 3 14:05:59 UTC 2020 x86_64 x86_64 x86_64 GNU/Linux'
  succeeded: true


# The stackstorm exchange
You can find over 170 prebuilt st2 integration packs on the echange. Let's have a look.

[StackStorm Exchange](http://exchange.stackstorm.com)

You might of seen this on the Exchange website. Installing a pack in st2 is very easy. 
**st2 pack install** <packname>. Maybe you want to automate something in VMware. You would have to write a bunch of python to make that happen, pyVmomi (a module written in python) would be your new best friend. Even so, you would still write a couple 1000 lines of code vs., **st2 pack install vsphere**.

# Finally, the stackstorm GUI
I promised you this from the beginning. It's handy, I find I use the GUI more to monitor the sensors, rules and actions. After I get it put together from the command line (creating YAML files is way easier with an editor), I make sure it's running properly from the GUI (So much easier than on the command line). Before we go we are going to need the st2admin user ID password we used to login from the command line earlier, located in the **fourth cell. Go copy it now, you can paste it into the login screen for the GUI.  

[StackStorm GUI](http://localhost)